#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [1]:
!pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable


In [2]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=18743 sha256=018bcde45bd6ea07ef1db1ba24d80cba1329bc4e04485978df85a2a0d9787f00
  Stored in directory: /tmp/pip-ephem-wheel-cache-uaryng98/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [3]:
import os
import pandas as pd

In [4]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNLP


2023-09-24 14:31:44.967950: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 14:31:45.002001: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 14:31:45.002557: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 14:31:45.488129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Get latest file URLS and Current Date Ranges

In [5]:
latest_URLS = Boston311LogReg.get311URLs()

In [6]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmph34r4avm.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [7]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")
tomorrow_datestring = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

print(today_datestring, thirty_days_ago_datestring, tomorrow_datestring)

2023-09-24 2023-08-25 2023-09-25


In [8]:
#set model folder constant
MODEL_FOLDER = './daily_models'

## Load extra features

In [9]:
#set path to mydata
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'


##Define several models

In [10]:
linear_tree_model = Boston311SurvDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type','queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [11]:
logistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [12]:
oldlogistic_model = Boston311LogReg(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [13]:
logistic_tree_model = Boston311EventDecTree(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0},
                            files_dict=latest_URLS)

In [14]:
kerasNLP_model = Boston311KerasNLP(train_date_range={'start':'2022-01-01','end':thirty_days_ago_datestring},
                            predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring},
                            feature_columns=['type', 'queue', 'source', 'subject', 'reason', 'department', 'ward_number'],
                            scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']},
                                      'survivalTimeMin':0,
                                      'survivalTimeFill':tomorrow_datestring},
                            files_dict=latest_URLS)

In [15]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [16]:
mydata = kerasNLP_model.load_data()



/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:259: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:259: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]


In [17]:
mydata['case_enquiry_id']

0         101004113346
1         101004113347
2         101004113348
3         101004113349
4         101004113351
              ...     
476127    101005012397
476128    101005012398
476129    101005012399
476130    101005012400
476131    101005012401
Name: case_enquiry_id, Length: 476132, dtype: int64

In [18]:
mydata = kerasNLP_model.enhance_data(mydata)


In [19]:
mydata = kerasNLP_model.apply_scenario(mydata)


In [20]:

mydata = kerasNLP_model.clean_data(mydata)

In [21]:
print(mydata['case_enquiry_id'])

0         101004113346
1         101004113347
2         101004113348
3         101004113349
4         101004113351
              ...     
476127    101005012397
476128    101005012398
476129    101005012399
476130    101005012400
476131    101005012401
Name: case_enquiry_id, Length: 423410, dtype: int64


In [22]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    X['cls_embedding'] = X['cls_embedding'].apply(literal_eval).apply(np.array)
    X['pooled_embedding'] = X['pooled_embedding'].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [23]:
X.shape

(275503, 3)

In [24]:
df = X
# Assuming df is your DataFrame and it has columns 'cls_embedding' and 'pooled_embedding'
cls_embedding_flattened = np.stack(df['cls_embedding'].to_numpy())
pooled_embedding_flattened = np.stack(df['pooled_embedding'].to_numpy())

# Remove the old columns
df.drop(['cls_embedding', 'pooled_embedding'], axis=1, inplace=True)

# Add the new flattened columns
df_cls = pd.DataFrame(cls_embedding_flattened, columns=[f'cls_{i}' for i in range(cls_embedding_flattened.shape[1])])
df_pooled = pd.DataFrame(pooled_embedding_flattened, columns=[f'pooled_{i}' for i in range(pooled_embedding_flattened.shape[1])])

df = pd.concat([df, df_cls, df_pooled], axis=1)
X = df

In [25]:
X['case_enquiry_id']

0               101004615710
1               101004615710
2         service_request_id
3               101004616099
4               101004616098
                 ...        
275498          101005063009
275499          101005062363
275500          101005061781
275501          101005063039
275502          101005062077
Name: case_enquiry_id, Length: 275503, dtype: object

In [26]:
X['case_enquiry_id'] = X['case_enquiry_id'].astype(str)
is_numeric = X['case_enquiry_id'].str.isnumeric()

In [27]:
X = X[is_numeric]

In [28]:
X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

/tmp/ipykernel_458459/2653176235.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')


In [29]:
X.shape

(275502, 257)

In [30]:
mydata.shape

(423410, 443)

In [31]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(X, on='case_enquiry_id', how='inner')


In [32]:
new_mydata.shape

(156025, 699)

In [33]:

X, y = kerasNLP_model.split_data(new_mydata)

In [34]:
#cast all columns that are type bool to float
for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype('float64')

In [35]:
#list the number of rows in X and y
print(X.shape)
print(y.shape)


(156025, 696)
(156025,)


In [36]:
best_model, best_hyperparameters = kerasNLP_model.tune_model(X, y, '/home/briarmoss/Documents/Boston_311/models/tuning')

Trial 26 Complete [00h 02m 06s]
val_accuracy: 0.7302035093307495

Best val_accuracy So Far: 0.735074520111084
Total elapsed time: 01h 24m 25s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
1024              |1024              |start_nodes
64                |64                |end_nodes
0.00013238        |1e-05             |l2_0
1e-05             |0.00014927        |learning_rate

Epoch 1/10
1055/3901 [=======>......................] - ETA: 23s - loss: 2.0726 - accuracy: 0.3526 - top_k_categorical_accuracy: 0.5366

3901/3901 [==============================] - 34s 9ms/step - loss: 1.7729 - accuracy: 0.4443 - top_k_categorical_accuracy: 0.6530 - val_loss: 1.5523 - val_accuracy: 0.5129 - val_top_k_categorical_accuracy: 0.7332
Epoch 2/10
2688/3901 [===================>..........] - ETA: 9s - loss: 1.4927 - accuracy: 0.5382 - top_k_categorical_accuracy: 0.7456

In [ ]:

#parse CLS embedding column as array
test_acc = kerasNLP_model.train_model( X, y )

Starting Training at 2023-09-24 00:55:45.726527


Epoch 1/100
3901/3901 [==============================] - 9s 2ms/step - loss: 1.5726 - accuracy: 0.5613 - top_k_categorical_accuracy: 0.7516 - val_loss: 1.3970 - val_accuracy: 0.5932 - val_top_k_categorical_accuracy: 0.7760
Epoch 2/100
3901/3901 [==============================] - 8s 2ms/step - loss: 1.2808 - accuracy: 0.6225 - top_k_categorical_accuracy: 0.7959 - val_loss: 1.2950 - val_accuracy: 0.6143 - val_top_k_categorical_accuracy: 0.7894
Epoch 3/100
3901/3901 [==============================] - 8s 2ms/step - loss: 1.1986 - accuracy: 0.6436 - top_k_categorical_accuracy: 0.8068 - val_loss: 1.2789 - val_accuracy: 0.6190 - val_top_k_categorical_accuracy: 0.7743
Epoch 4/100
3901/3901 [==============================] - 8s 2ms/step - loss: 1.1547 - accuracy: 0.6560 - top_k_categorical_accuracy: 0.8152 - val_loss: 1.1756 - val_accuracy: 0.6395 - val_top_k_categorical_accuracy: 0.8143
Epoch 5/100
3901/3901 [==============================] - 8s 2ms/step - loss: 1.1288 - accuracy: 0.6632 - top

## Train several models

In [ ]:
print("learning is fun!") 

learning is fun!


In [ ]:
#logistic_tree_model.run_pipeline()

In [ ]:
#logistic_model.run_pipeline()

In [ ]:
import gc
gc.collect()

2412

In [ ]:
#linear_tree_model.run_pipeline()

In [ ]:
"""
import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)

# List of models
models = [linear_tree_model, logistic_tree_model, logistic_model]


# Iterate over models and save
for model in models:
    save_model_to_dir(model, model.model_type)
"""


'\nimport datetime\n\ndef save_model_to_dir(model, folder_name):\n    dir_path = os.path.join(MODEL_FOLDER, folder_name)\n    \n    if not os.path.exists(dir_path):\n        os.mkdir(dir_path)\n    \n    timestamp = datetime.datetime.now().strftime(\'%Y%m%d_%H%M%S\')\n    model_name = timestamp + "_" + model.model_type\n    properties_name = model_name\n    \n    model.save(dir_path, model_name, properties_name)\n\n# List of models\nmodels = [linear_tree_model, logistic_tree_model, logistic_model]\n\n\n# Iterate over models and save\nfor model in models:\n    save_model_to_dir(model, model.model_type)\n'